# Taxi Exercise

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [ ]:
ls -l # check the .tgz is there

total 267680
drwxr-xr-x 1 root root      4096 Dec  8 14:36 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [ ]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [ ]:
!pip install -q findspark

Defining the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Window Partitioning Exercises") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/taxi_data.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/taxi_zones.csv -P /dataset

## Load datasets

In [ ]:
taxiDF = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/dataset/taxi_data.csv")

taxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
taxiZonesDF = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/dataset/taxi_zones.csv")

taxiZonesDF.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



## Taxi Exercise

In this exercise we will be working with two DFs. The first one, taxiDf holds info about taxi rides per 2018 year. And the second, taxiZonesDF, have info about the Zones. Please load the DFs and print the schemas and two (or more) rows for more detailed info.

The aim of the exercise is to answer the questions listed below.

## Questions:

 1. Which zones have the most pickups/dropoffs overall? Note there are many PULocationIDs per Zone?
 2. What are the peak hours for taxi?
 3. How are the trips distributed by length? Show stats like mean, max, min, etc. 
    Then get the total trips for less/more than 30 km. Why are people taking the cab? For long or short trips?
    You can also try the same with different distances. Which is the expected value for threshold is we want to obtain more or less the same trips in long/short counting?
 4. What are the peak hours for long/short trips?
 5. What are the top 3 pickup/dropoff zones for long/short trips?
 6. How are people paying for the ride, on long/short trips? Hint: the information about how good is the payment is in RatecodeID column.
 7. How is the payment type (RatecodeId) evolving with time (in days)? Hint: use the column with pickup time info.
    Get the same info but with avg of ratecode and total trips per day.